# Limpeza dos dados

In [1]:
import pandas as pd
import janitor as jn

In [2]:
df = pd.DataFrame(
    {
        "A": [1, None, 3, ],
        " Coluna b ": [3.7, 10, None]
    }
)

In [3]:
df

,A,Coluna b
0,1.0,3.7
1,NaN,10.0
2,3.0,NaN


A função `clean_data` recebe um dataframe e retorna um dataframe com os dados limpos. Basicamente o que ela faz é remover os espaços dos nomes das colunas e adicioanr '_' no lugar dos espaços e remover caracteres maiúsculos dos nomes das colunas.

In [4]:
clean_df = jn.clean_names(df)

In [5]:
print(f'DataFrame original: \n\n{df}\n{20*"-"}\nDataFrame limpo:\n\n{clean_df}')

DataFrame original: 

     A   Coluna b 
0  1.0         3.7
1  NaN        10.0
2  3.0         NaN
--------------------
DataFrame limpo:

     a  _coluna_b_
0  1.0         3.7
1  NaN        10.0
2  3.0         NaN


É recomendável atualizar as colunas usando atribuição com índices, o métdo `assign` ou atribuições com `.loc` ou `.iloc`.

O método `assign` funciona da seguinte forma:

```python
df = df.assign(coluna1 = df['coluna1'].str.replace(' ', '_').str.lower())
```

O que ele faz é criar uma nova coluna chamada `coluna1` com os dados da coluna `coluna1` com os espaços substituídos por `_` e os caracteres maiúsculos substituídos por minúsculos.


In [6]:
def clean_name(name):
    return(
        name.strip().lower().replace(" ", "_") # strip() remove espaços em branco no início e no fim da string, replace() substitui um caractere por outro
    )

In [7]:
df.rename(columns=clean_name)

,a,coluna_b
0,1.0,3.7
1,NaN,10.0
2,3.0,NaN


In [8]:
print(f'Limpeza usando o pyjanitor: \n\n{clean_df}\n{20*"-"}\nLimpeza usando o pandas:\n\n{df.rename(columns=clean_name)}')

Limpeza usando o pyjanitor: 

     a  _coluna_b_
0  1.0         3.7
1  NaN        10.0
2  3.0         NaN
--------------------
Limpeza usando o pandas:

     a  coluna_b
0  1.0       3.7
1  NaN      10.0
2  3.0       NaN


Como pode ser observado, por mais que o `pyjanitor` consiga fazer uma limpeza interessante, é necessário um conhecimento básico de manipulação de dados para que se possa fazer uma limpeza mais completa.

Utilizando a função `coalesce` do `pyjanitor` é possível substituir os valores nulos por um valor padrão. No caso, se passarmos duas colunas para a função, ela irá substituir os valores nulos da primeira coluna pelos valores da segunda coluna.

In [28]:
jn.coalesce(
    df,
    "A",
    " Coluna b ",
    new_column_name="val"
)

/home/pcmoraes/anaconda3/lib/python3.11/site-packages/janitor/utils.py:365: DeprecationWarning: new_column_name is deprecated; use target_column_name
  warn(


,A,Coluna b,val
0,1.0,3.7,1.0
1,NaN,10.0,10.0
2,3.0,NaN,3.0


Entretanto, a substituição de valores nulos pode ser feito através de um valor especifico utilizando o método `fillna` do pandas.


In [30]:
df.fillna(0) # Substitui os valores nulos por 0

,A,Coluna b
0,1.0,3.7
1,0.0,10.0
2,3.0,0.0


Ou utilizando a função `fill_empty` do `pyjanitor`:

In [34]:
jn.fill_empty(
    df,
    column_names=["A", " Coluna b "],
    value=0
)


/tmp/ipykernel_46392/151377894.py:1: FutureWarning: This function will be deprecated in a 1.x release. Kindly use `jn.impute` instead.
  jn.fill_empty(


,A,Coluna b
0,1.0,3.7
1,0.0,10.0
2,3.0,0.0


Como a função `fill_empty` será removida do `pyjanitor`, pode ser utilizado a função `impute` do `pyjanitor`

In [35]:
jn.impute(
    df,
    column_names=["A", " Coluna b "],
    value=0
)


,A,Coluna b
0,1.0,3.7
1,0.0,10.0
2,3.0,0.0


Com frequência a substituição de dados é feita utilizando métodos do `scikit-learn` ou do `fancyimpute`

In [38]:
from sklearn.impute import SimpleImputer


In [39]:

imputer = SimpleImputer(strategy="mean") # Existem diversas estratégias para imputação de dados faltantes, como a média, mediana, moda, etc.
imputacao = imputer.fit_transform(df[["A", " Coluna b "]])

print(f'Valores originais: \n\n{df}\n{20*"-"}\nValores imputados:\n\n{imputacao}')

Valores originais: 

     A   Coluna b 
0  1.0         3.7
1  NaN        10.0
2  3.0         NaN
--------------------
Valores imputados:

[[ 1.    3.7 ]
 [ 2.   10.  ]
 [ 3.    6.85]]


In [43]:
from fancyimpute import IterativeImputer

A estratégia utilizada pelo modelo `IterativeImputer` do `fancyimpute` é a seguinte:

Ele utiliza um modelo de regressão para prever os valores nulos de uma coluna baseado nos valores não nulos da mesma coluna e nas outras colunas do dataframe. A cada iteração ele utiliza um modelo diferente para prever os valores nulos. O número de iterações é definido pelo usuário. O modelo linear é separado para cada uma das colunas.

In [52]:
imputer = IterativeImputer(max_iter=0) # Pode ser feito o ajuste do maximo de iterações, quando é definada como 0, ele retorna a média
imputacao = imputer.fit_transform(df[["A", " Coluna b "]])
print(f'Valores originais: \n\n{df}\n{20*"-"}\nValores imputados:\n\n{imputacao}')

Valores originais: 

     A   Coluna b 
0  1.0         3.7
1  NaN        10.0
2  3.0         NaN
--------------------
Valores imputados:

[[ 1.    3.7 ]
 [ 2.   10.  ]
 [ 3.    6.85]]


Antes de iniciar a construção do modelo, é recomendável fazer uma verificação para assegurar que todos os dados ausentes foram tratados.

In [55]:
df.isna().any().any() # Retorna True se houver algum valor nulo no DataFrame

True

In [59]:
imputacao = pd.DataFrame(imputacao, columns=["A", " Coluna b "])
imputacao.isna().any().any() # Retorna False se não houver nenhum valor nulo no DataFrame

False

In [60]:
imputacao

,A,Coluna b
0,1.0,3.70
1,2.0,10.00
2,3.0,6.85
